In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
labels_path = '/content/drive/MyDrive/Proyecto_integrador/dataset.csv'
labels_df = pd.read_csv(labels_file, sep=';')
image_dir = '/content/drive/MyDrive/Proyecto_integrador/Imagenes'

In [ ]:
image_paths = []
image_labels = []

for filename in os.listdir(image_dir):
    if filename.endswith('.png'):  
        image_path = os.path.join(image_dir, filename)
        matching_labels = labels_df[labels_df['source'] == filename]['target'].values
        if len(matching_labels) > 0:
            label = matching_labels[0]
            image_paths.append(image_path)
            image_labels.append(label)
        else:
            print(f"No se encontró etiqueta para el archivo: {filename}. Omitiendo esta imagen.")

In [ ]:
resnet = models.resnet50(pretrained=True)
resnet = resnet.eval()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

In [ ]:
def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    features = resnet(image)
    features = features.squeeze().detach().numpy()
    return features

In [ ]:
features = []
for image_path in image_paths:
    feature = extract_features(image_path)
    features.append(feature)
features = np.array(features)